# Topic 10: Advanced Patterns

Master production-ready patterns and best practices for building robust, scalable LangGraph applications. Learn error handling, monitoring, optimization, and advanced architectural patterns.

## Learning Objectives

- Implement error handling and recovery
- Add monitoring and logging
- Optimize graph performance
- Use advanced architectural patterns
- Build production-ready applications

In [ ]:
# Setup
import os
import getpass
from typing import TypedDict, Literal, Optional
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver
from langchain_anthropic import ChatAnthropic
from langchain_core.messages import HumanMessage, SystemMessage
import logging
from datetime import datetime
import time

if "ANTHROPIC_API_KEY" not in os.environ:
    os.environ["ANTHROPIC_API_KEY"] = getpass.getpass("Enter your Anthropic API key: ")

model = ChatAnthropic(model="claude-sonnet-4-20250514")

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

print("✓ Setup complete!")

## Pattern 1: Error Handling and Recovery

Robust applications must handle errors gracefully and recover when possible.

In [ ]:
class RobustState(TypedDict):
    input: str
    result: str
    error: Optional[str]
    retry_count: int
    max_retries: int

def risky_operation(state: RobustState) -> RobustState:
    """Operation that might fail - with error handling."""
    print(f"\n🔄 Attempt {state['retry_count'] + 1}/{state['max_retries']}")
    
    try:
        # Simulate potential failure
        if state['retry_count'] < 2:  # Fail first 2 times for demo
            raise Exception("Simulated API error")
        
        # Actual operation
        response = model.invoke([HumanMessage(content=state['input'])])
        
        logger.info("Operation succeeded")
        return {
            "result": response.content,
            "error": None,
            "retry_count": state['retry_count'] + 1
        }
        
    except Exception as e:
        logger.error(f"Operation failed: {str(e)}")
        return {
            "error": str(e),
            "retry_count": state['retry_count'] + 1
        }

def error_handler(state: RobustState) -> RobustState:
    """Handle errors with fallback logic."""
    print("\n⚠️  Error handler activated")
    logger.warning(f"Handling error: {state['error']}")
    
    return {
        "result": f"Operation failed after {state['retry_count']} attempts. Error: {state['error']}"
    }

def should_retry(state: RobustState) -> Literal["retry", "error", "success"]:
    """Decide whether to retry, handle error, or succeed."""
    if state['error'] is None:
        print("✅ Success!")
        return "success"
    elif state['retry_count'] < state['max_retries']:
        print("🔄 Retrying...")
        time.sleep(1)  # Backoff
        return "retry"
    else:
        print("❌ Max retries reached")
        return "error"

print("✓ Error handling nodes created")

In [ ]:
# Build robust graph with error handling
robust_builder = StateGraph(RobustState)

robust_builder.add_node("operation", risky_operation)
robust_builder.add_node("error_handler", error_handler)

robust_builder.add_edge(START, "operation")

robust_builder.add_conditional_edges(
    "operation",
    should_retry,
    {
        "retry": "operation",  # Loop back for retry
        "error": "error_handler",
        "success": END
    }
)

robust_builder.add_edge("error_handler", END)

robust_graph = robust_builder.compile()

print("✓ Robust graph compiled!")

In [ ]:
# Test error handling and retry logic
print("Testing error handling...")
print("="*60)

result = robust_graph.invoke({
    "input": "What is LangGraph?",
    "result": "",
    "error": None,
    "retry_count": 0,
    "max_retries": 5
})

print("\n" + "="*60)
print("Final Result:")
print(result['result'][:200] if len(result['result']) > 200 else result['result'])

## Pattern 2: Monitoring and Observability

Add comprehensive logging and metrics to track graph execution.

In [ ]:
class MonitoredState(TypedDict):
    input: str
    output: str
    metrics: dict

def monitored_node(state: MonitoredState) -> MonitoredState:
    """Node with comprehensive monitoring."""
    start_time = time.time()
    
    logger.info(f"Processing input: {state['input'][:50]}...")
    
    try:
        response = model.invoke([HumanMessage(content=state['input'])])
        
        execution_time = time.time() - start_time
        
        metrics = {
            "execution_time": execution_time,
            "timestamp": datetime.now().isoformat(),
            "input_length": len(state['input']),
            "output_length": len(response.content),
            "success": True
        }
        
        logger.info(f"Execution time: {execution_time:.2f}s")
        logger.info(f"Output length: {len(response.content)} chars")
        
        return {
            "output": response.content,
            "metrics": metrics
        }
        
    except Exception as e:
        execution_time = time.time() - start_time
        
        metrics = {
            "execution_time": execution_time,
            "timestamp": datetime.now().isoformat(),
            "error": str(e),
            "success": False
        }
        
        logger.error(f"Node failed: {str(e)}")
        
        return {
            "output": f"Error: {str(e)}",
            "metrics": metrics
        }

print("✓ Monitored node created")

In [ ]:
# Build monitored graph
monitored_builder = StateGraph(MonitoredState)
monitored_builder.add_node("process", monitored_node)
monitored_builder.add_edge(START, "process")
monitored_builder.add_edge("process", END)

monitored_graph = monitored_builder.compile()

# Test with monitoring
result = monitored_graph.invoke({
    "input": "Explain LangGraph in simple terms",
    "output": "",
    "metrics": {}
})

print("\nMetrics:")
for key, value in result['metrics'].items():
    print(f"  {key}: {value}")

## Pattern 3: Dynamic Routing with Confidence Scores

Make routing decisions based on confidence levels and quality metrics.

In [ ]:
class ConfidenceState(TypedDict):
    query: str
    response: str
    confidence: float
    fallback_used: bool

def primary_agent(state: ConfidenceState) -> ConfidenceState:
    """Primary agent with confidence scoring."""
    print("\n🎯 Primary agent processing...")
    
    prompt = f"{state['query']}\n\nRate your confidence in this answer from 0-1."
    
    response = model.invoke([HumanMessage(content=state['query'])])
    
    # Simulate confidence calculation (in practice, use actual metrics)
    confidence = 0.85 if len(response.content) > 100 else 0.6
    
    print(f"   Confidence: {confidence}")
    
    return {
        "response": response.content,
        "confidence": confidence,
        "fallback_used": False
    }

def fallback_agent(state: ConfidenceState) -> ConfidenceState:
    """Fallback agent for low-confidence scenarios."""
    print("\n🔄 Fallback agent activated...")
    
    prompt = f"""The primary agent was uncertain. Please provide a very careful, 
    well-researched answer to: {state['query']}"""
    
    response = model.invoke([HumanMessage(content=prompt)])
    
    return {
        "response": response.content,
        "confidence": 0.9,  # Fallback is more thorough
        "fallback_used": True
    }

def route_by_confidence(state: ConfidenceState) -> Literal["fallback", "success"]:
    """Route based on confidence threshold."""
    threshold = 0.75
    
    if state['confidence'] < threshold:
        print(f"⚠️  Low confidence ({state['confidence']}) - using fallback")
        return "fallback"
    else:
        print(f"✅ High confidence ({state['confidence']})")
        return "success"

print("✓ Confidence routing nodes created")

In [ ]:
# Build confidence-based routing graph
confidence_builder = StateGraph(ConfidenceState)

confidence_builder.add_node("primary", primary_agent)
confidence_builder.add_node("fallback", fallback_agent)

confidence_builder.add_edge(START, "primary")

confidence_builder.add_conditional_edges(
    "primary",
    route_by_confidence,
    {
        "fallback": "fallback",
        "success": END
    }
)

confidence_builder.add_edge("fallback", END)

confidence_graph = confidence_builder.compile()

print("✓ Confidence routing graph compiled!")

In [ ]:
# Test confidence routing
print("Test 1: Simple query (high confidence expected)")
print("="*60)

result1 = confidence_graph.invoke({
    "query": "What is Python?",
    "response": "",
    "confidence": 0.0,
    "fallback_used": False
})

print(f"\nUsed fallback: {result1['fallback_used']}")
print(f"Final confidence: {result1['confidence']}")

## Pattern 4: Circuit Breaker Pattern

Protect your system from cascading failures with circuit breakers.

In [ ]:
class CircuitBreakerState(TypedDict):
    request: str
    response: str
    failure_count: int
    circuit_open: bool
    threshold: int

def protected_operation(state: CircuitBreakerState) -> CircuitBreakerState:
    """Operation protected by circuit breaker."""
    
    if state.get('circuit_open', False):
        logger.warning("Circuit breaker is OPEN - rejecting request")
        return {
            "response": "Service temporarily unavailable",
            "circuit_open": True
        }
    
    try:
        print("\n⚡ Executing protected operation...")
        
        # Simulate operation that might fail
        response = model.invoke([HumanMessage(content=state['request'])])
        
        logger.info("Operation succeeded - resetting failure count")
        return {
            "response": response.content,
            "failure_count": 0,  # Reset on success
            "circuit_open": False
        }
        
    except Exception as e:
        new_failure_count = state.get('failure_count', 0) + 1
        logger.error(f"Operation failed ({new_failure_count}/{state['threshold']})")
        
        # Open circuit if threshold reached
        circuit_open = new_failure_count >= state['threshold']
        
        if circuit_open:
            logger.critical("CIRCUIT BREAKER OPENED")
        
        return {
            "response": f"Error: {str(e)}",
            "failure_count": new_failure_count,
            "circuit_open": circuit_open
        }

print("✓ Circuit breaker pattern implemented")

## Pattern 5: Caching and Optimization

Optimize performance with intelligent caching.

In [ ]:
# Simple in-memory cache
response_cache = {}

class CachedState(TypedDict):
    query: str
    response: str
    cache_hit: bool

def cached_operation(state: CachedState) -> CachedState:
    """Operation with caching."""
    query = state['query']
    
    # Check cache first
    if query in response_cache:
        print("\n✨ Cache HIT")
        logger.info(f"Cache hit for query: {query[:50]}")
        return {
            "response": response_cache[query],
            "cache_hit": True
        }
    
    # Cache miss - execute operation
    print("\n🔄 Cache MISS - executing...")
    logger.info(f"Cache miss for query: {query[:50]}")
    
    response = model.invoke([HumanMessage(content=query)])
    
    # Store in cache
    response_cache[query] = response.content
    
    return {
        "response": response.content,
        "cache_hit": False
    }

print("✓ Caching implemented")

In [ ]:
# Build cached graph
cached_builder = StateGraph(CachedState)
cached_builder.add_node("process", cached_operation)
cached_builder.add_edge(START, "process")
cached_builder.add_edge("process", END)

cached_graph = cached_builder.compile()

# Test caching
print("Test 1: First request (cache miss)")
result1 = cached_graph.invoke({
    "query": "What is caching?",
    "response": "",
    "cache_hit": False
})

print(f"\nCache hit: {result1['cache_hit']}")

print("\n" + "="*60)
print("Test 2: Same request (cache hit)")
result2 = cached_graph.invoke({
    "query": "What is caching?",
    "response": "",
    "cache_hit": False
})

print(f"\nCache hit: {result2['cache_hit']}")
print(f"Response matches: {result1['response'] == result2['response']}")

## Pattern 6: Rate Limiting

Control API usage and prevent overload with rate limiting.

In [ ]:
from collections import deque
import time

class RateLimiter:
    def __init__(self, max_calls: int, time_window: int):
        self.max_calls = max_calls
        self.time_window = time_window
        self.calls = deque()
    
    def allow_request(self) -> bool:
        now = time.time()
        
        # Remove old calls outside window
        while self.calls and self.calls[0] < now - self.time_window:
            self.calls.popleft()
        
        # Check if under limit
        if len(self.calls) < self.max_calls:
            self.calls.append(now)
            return True
        
        return False

# Create rate limiter: 3 calls per 10 seconds
rate_limiter = RateLimiter(max_calls=3, time_window=10)

class RateLimitedState(TypedDict):
    request: str
    response: str
    rate_limited: bool

def rate_limited_operation(state: RateLimitedState) -> RateLimitedState:
    """Operation with rate limiting."""
    
    if not rate_limiter.allow_request():
        print("\n🚫 Rate limit exceeded")
        logger.warning("Request rate limited")
        return {
            "response": "Rate limit exceeded. Please try again later.",
            "rate_limited": True
        }
    
    print("\n✅ Request allowed")
    response = model.invoke([HumanMessage(content=state['request'])])
    
    return {
        "response": response.content,
        "rate_limited": False
    }

print("✓ Rate limiting implemented")

## Best Practices Summary

### 1. Error Handling
- Always wrap operations in try-except blocks
- Implement retry logic with exponential backoff
- Provide meaningful error messages
- Have fallback strategies

### 2. Monitoring
- Log all important events
- Track execution times
- Monitor success/failure rates
- Record metrics for analysis

### 3. Performance
- Cache frequently accessed data
- Use rate limiting to protect services
- Implement circuit breakers for resilience
- Optimize node execution order

### 4. Architecture
- Keep nodes focused and single-purpose
- Use clear state definitions
- Implement proper separation of concerns
- Make graphs testable

### 5. Production Readiness
- Add comprehensive testing
- Implement health checks
- Use proper configuration management
- Document your graphs

## Exercise: Build a Production-Ready Graph

Combine multiple patterns to create a robust system:
1. Add error handling with retries
2. Implement monitoring and logging
3. Add caching for performance
4. Include rate limiting
5. Use confidence-based routing

Create a question-answering system with all these features.

In [ ]:
# Your code here!

class ProductionState(TypedDict):
    question: str
    answer: str
    confidence: float
    cache_hit: bool
    retry_count: int
    error: Optional[str]
    metrics: dict

# TODO: Implement nodes that combine:
# - Caching
# - Rate limiting
# - Error handling with retries
# - Monitoring
# - Confidence-based routing

# TODO: Build the graph
# TODO: Test with various scenarios

## Key Takeaways

In this notebook, you learned:

1. ✅ Implementing robust error handling and recovery
2. ✅ Adding comprehensive monitoring and logging
3. ✅ Using confidence scores for intelligent routing
4. ✅ Implementing circuit breaker patterns
5. ✅ Optimizing with caching strategies
6. ✅ Adding rate limiting for protection
7. ✅ Following production-ready best practices

## Congratulations!

You've completed the LangGraph training! You now have the knowledge to build:
- Simple chatbots
- Complex multi-agent systems
- Production-ready workflows
- Robust, scalable applications

## Next Steps

- Build your own LangGraph application
- Explore the [official documentation](https://langchain-ai.github.io/langgraph/)
- Join the community and share your projects
- Keep learning and experimenting!

Happy building! 🚀